# Loading Dataset

In [2]:
import requests
import pandas as pd
import time
import duckdb

!pip install duckdb

In [445]:
DATASET_ID = "2f1e57ea-ac2f-4f62-aeb1-f8254307c395" 
BASE_URL = f"https://data.cms.gov/data-api/v1/dataset/{DATASET_ID}/data"

def fetch_full_medicare_dataset(total_rows=146427, batch_size=5000):
    all_data = []
    offset = 0
    while offset < total_rows:
        params = {
            'size': batch_size,
            'offset': offset
        }
        
        try:
            response = requests.get(BASE_URL, params=params, timeout=30)
            response.raise_for_status() # Check for errors
            
            data = response.json()
            if not data:
                break
                
            all_data.extend(data)
            offset += batch_size
        
            progress = min(len(all_data), total_rows)
            print(f"Progress: {progress}/{total_rows} rows downloaded...", end="\r")
            time.sleep(0.5) 
            
        except Exception as e:
            print(f"\n Error at offset {offset}: {e}")
            break

    print(f"\n\n Done! Total rows collected: {len(all_data)}")
    return pd.DataFrame(all_data)

df = fetch_full_medicare_dataset()

df.to_csv("medicare_inpatient_full_2023.csv", index=False)

Starting download of 146427 rows...
Progress: 146427/146427 rows downloaded...

 Done! Total rows collected: 146427


In [446]:
df.head()

,Rndrng_Prvdr_CCN,Rndrng_Prvdr_Org_Name,Rndrng_Prvdr_City,Rndrng_Prvdr_St,Rndrng_Prvdr_State_FIPS,Rndrng_Prvdr_Zip5,Rndrng_Prvdr_State_Abrvtn,Rndrng_Prvdr_RUCA,Rndrng_Prvdr_RUCA_Desc,DRG_Cd,DRG_Desc,Tot_Dschrgs,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",003,"ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DIAGNOSIS EXCEPT FACE, MOUTH AND NEC",14,663764.35714,120219.92857,115544.14286
1,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE COMPLEX CNS PRINCIPAL DIAGNOSIS WITH MCC O,26,180980.88462,37321.038462,35261.807692
2,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE COMPLEX CNS PRINCIPAL DIAGNOSIS WITHOUT MC,12,105824.33333,26936.666667,25048.916667
3,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",025,CRANIOTOMY AND ENDOVASCULAR INTRACRANIAL PROCEDURES WITH MCC,16,242539.5,34745.375,32438.625
4,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",038,EXTRACRANIAL PROCEDURES WITH CC,11,122741.18182,14999.818182,9579.3636364


# Data Cleaning

In [448]:
df.columns = [col.replace('Rndrng_Prvdr_', '') for col in df.columns]
print("Updated Columns:", df.columns.tolist())
df.head(1)

Updated Columns: ['CCN', 'Org_Name', 'City', 'St', 'State_FIPS', 'Zip5', 'State_Abrvtn', 'RUCA', 'RUCA_Desc', 'DRG_Cd', 'DRG_Desc', 'Tot_Dschrgs', 'Avg_Submtd_Cvrd_Chrg', 'Avg_Tot_Pymt_Amt', 'Avg_Mdcr_Pymt_Amt']


,CCN,Org_Name,City,St,State_FIPS,Zip5,State_Abrvtn,RUCA,RUCA_Desc,DRG_Cd,DRG_Desc,Tot_Dschrgs,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",003,"ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DIAGNOSIS EXCEPT FACE, MOUTH AND NEC",14,663764.35714,120219.92857,115544.14286


In [587]:
df.isnull().sum()

Provider_ID               0
Hospital_Name             0
City                      0
Address                   0
State_Code                0
Zip                       0
State                     0
Rural_Urban_Code          0
Region_Type               0
DRG_Code                  0
DRG_Description           0
Total_Discharges          0
Avg_Submtd_Cvrd_Chrg      0
Avg_Tot_Pymt_Amt          0
Avg_Mdcr_Pymt_Amt         0
MDC_Code                  0
Service_Line              0
Avg_Patient_OOP           0
Patient_Share_Pct         0
Reimbursement_Rate_Pct    0
Region_Category           0
dtype: int64

In [449]:
rename_map = {
    'CCN': 'Provider_ID',
    'Org_Name': 'Hospital_Name',
    'City': 'City',
    'St': 'Address',
    'State_FIPS': 'State_Code',
    'Zip5': 'Zip',
    'State_Abrvtn': 'State',
    'RUCA': 'Rural_Urban_Code',
    'RUCA_Desc': 'Region_Type',
    'DRG_Cd': 'DRG_Code',
    'DRG_Desc': 'DRG_Description',
    'Tot_Dschrgs': 'Total_Discharges'
}

df.rename(columns=rename_map, inplace=True)
print(df.columns.tolist())
df.head()

['Provider_ID', 'Hospital_Name', 'City', 'Address', 'State_Code', 'Zip', 'State', 'Rural_Urban_Code', 'Region_Type', 'DRG_Code', 'DRG_Description', 'Total_Discharges', 'Avg_Submtd_Cvrd_Chrg', 'Avg_Tot_Pymt_Amt', 'Avg_Mdcr_Pymt_Amt']


,Provider_ID,Hospital_Name,City,Address,State_Code,Zip,State,Rural_Urban_Code,Region_Type,DRG_Code,DRG_Description,Total_Discharges,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",003,"ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DIAGNOSIS EXCEPT FACE, MOUTH AND NEC",14,663764.35714,120219.92857,115544.14286
1,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE COMPLEX CNS PRINCIPAL DIAGNOSIS WITH MCC O,26,180980.88462,37321.038462,35261.807692
2,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE COMPLEX CNS PRINCIPAL DIAGNOSIS WITHOUT MC,12,105824.33333,26936.666667,25048.916667
3,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",025,CRANIOTOMY AND ENDOVASCULAR INTRACRANIAL PROCEDURES WITH MCC,16,242539.5,34745.375,32438.625
4,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",038,EXTRACRANIAL PROCEDURES WITH CC,11,122741.18182,14999.818182,9579.3636364


In [450]:
# List of columns that should be Numeric (Floats)
float_cols = ['Avg_Submtd_Cvrd_Chrg', 'Avg_Tot_Pymt_Amt', 'Avg_Mdcr_Pymt_Amt']

# List of columns that should be Integers
int_cols = ['Total_Discharges']

# List of columns that should be Strings/Categorical
str_cols = ['Provider_ID', 'DRG_Code', 'MDC_Code', 'State', 'Zip']

# 1. Clean and convert Floats
for col in float_cols:
    if col in df.columns:
        # Remove commas/currency symbols if present, then convert to float
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(r'[$,]', '', regex=True), errors='coerce')

# 2. Clean and convert Integers
for col in int_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col].astype(str).str.replace(r'[,]', '', regex=True), errors='coerce').fillna(0).astype(int)

# 3. Ensure IDs and Codes stay as strings (to keep leading zeros)
for col in str_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.zfill(3 if 'Code' in col else 1)

print(df.dtypes)

Data types successfully updated:
Provider_ID              object
Hospital_Name            object
City                     object
Address                  object
State_Code               object
Zip                      object
State                    object
Rural_Urban_Code         object
Region_Type              object
DRG_Code                 object
DRG_Description          object
Total_Discharges          int64
Avg_Submtd_Cvrd_Chrg    float64
Avg_Tot_Pymt_Amt        float64
Avg_Mdcr_Pymt_Amt       float64
dtype: object


# Mapping the DRG_Code with Major Diagnosis Group(MDC) 
Link: "https://www.medicareinformatics.com/DiagnosisRelatedGroups/Tables"

In [451]:
df.head(1)

,Provider_ID,Hospital_Name,City,Address,State_Code,Zip,State,Rural_Urban_Code,Region_Type,DRG_Code,DRG_Description,Total_Discharges,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",003,"ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DIAGNOSIS EXCEPT FACE, MOUTH AND NEC",14,663764.35714,120219.92857,115544.14286


In [452]:
url = "https://www.medicareinformatics.com/DiagnosisRelatedGroups/Tables"
tables = pd.read_html(url)

# 1. Look for the table that actually contains DRG information
mdc_map = None
for t in tables:
    if 'Diagnosis-Related Group' in t.columns:
        mdc_map = t
        break

if mdc_map is not None:
    # 2. Keep only what we need and rename
    # Note: Some tables have an empty 'Unnamed' column at the end, so we select by position or name
    mdc_map = mdc_map[['Diagnosis-Related Group', 'Major Diagnostic Category', 'Description']].copy()
    mdc_map.columns = ['DRG_Code', 'MDC_Code', 'DRG_Description']
    
    # 3. Clean the Data Types
    # The scraped DRG_Code might be '1', but your 'df' might have '001'.
    mdc_map['DRG_Code'] = mdc_map['DRG_Code'].astype(str).str.zfill(3)
    
    print("Success! Official MDC map created.")
    display(mdc_map.head(5))
else:
    print("Could not find the DRG table. Try printing tables[1].columns to see what was found.")

Success! Official MDC map created.


,DRG_Code,MDC_Code,DRG_Description
0,001,NaN,Heart Transplant or Implant of Heart Assist System with MCC
1,002,NaN,Heart Transplant or Implant of Heart Assist System without MCC
2,003,NaN,"ECMO or Tracheostomy with MV >96 Hours or Principal Diagnosis Except Face, Mouth and Neck with Major O.R. Procedures"
3,004,NaN,"Tracheostomy with MV >96 Hours or Principal Diagnosis Except Face, Mouth and Neck without Major O.R. Procedures"
4,005,NaN,Liver Transplant with MCC or Intestinal Transplant


In [453]:
# 1. Fill the NaNs for Transplants
mdc_map['MDC_Code'] = mdc_map['MDC_Code'].fillna('00')

# 2. Clean the MDC_Code column
# If the site has '01 - Nervous System', we just want the '01'
mdc_map['MDC_Code'] = mdc_map['MDC_Code'].astype(str).str.extract('(\d+)').fillna('00')

# 3. Final alignment of DRG_Codes
# This ensures '1' becomes '001' to match your main hospital data
mdc_map['DRG_Code'] = mdc_map['DRG_Code'].astype(str).str.strip().str.zfill(3)

# 4. Drop duplicates (sometimes tables have repeated headers)
mdc_map = mdc_map.drop_duplicates(subset=['DRG_Code'])

print("Cleaned Map Preview:")
display(df.head(1)) # Looking at the first 20 to see the transition from Pre-MDC to MDC 01

Cleaned Map Preview:


,Provider_ID,Hospital_Name,City,Address,State_Code,Zip,State,Rural_Urban_Code,Region_Type,DRG_Code,DRG_Description,Total_Discharges,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt
0,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",003,"ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DIAGNOSIS EXCEPT FACE, MOUTH AND NEC",14,663764.35714,120219.92857,115544.14286


In [454]:
# 1. Clean the main df DRG codes just like we did for the map
# This removes any '.0' or whitespace to ensure a perfect match
df['DRG_Code'] = df['DRG_Code'].astype(str).str.split('.').str[0].str.strip().str.zfill(3)

# 2. Drop the old MDC_Code column if it exists to keep the dataframe clean
if 'MDC_Code' in df.columns:
    df = df.drop(columns=['MDC_Code'])

# 3. Merge the cleaned map into your main df
# We keep only the columns we need from the map
df = df.merge(mdc_map[['DRG_Code', 'MDC_Code']], on='DRG_Code', how='left')

# 4. Verification
unmapped_count = df['MDC_Code'].isna().sum()
print(f"Enrichment Complete. Rows failed to map: {unmapped_count}")

Enrichment Complete. Rows failed to map: 4465


In [455]:
# 1. Isolate the failures
failed_rows = df[df['MDC_Code'].isna()]

print("--- FAILURE ANALYSIS ---")
print(f"Total Rows in Main DF: {len(df)}")
print(f"Total Failed Rows: {len(failed_rows)}")

# 2. Check the unique DRG codes that are failing
failed_codes = failed_rows['DRG_Code'].unique()
print(f"\nALL Unique DRG Codes that failed to map : {failed_codes}")

# 3. Check for hidden characters (The most likely culprit)
if len(failed_codes) > 0:
    sample_code = failed_codes[0]
    print(f"\nRepresentation of a failed code '{sample_code}': {repr(sample_code)}")
    
# 4. Check the Mapping Table (mdc_map)
print(f"\nRows in mdc_map: {len(mdc_map)}")
if len(mdc_map) > 0:
    print(f"Sample DRG from mdc_map: {repr(mdc_map['DRG_Code'].iloc[0])}")

--- FAILURE ANALYSIS ---
Total Rows in Main DF: 146427
Total Failed Rows: 4465

ALL Unique DRG Codes that failed to map : ['246' '247' '454' '455' '460' '459' '453' '227' '226' '078' '342' '343'
 '222' '225' '339' '077']

Representation of a failed code '246': '246'

Rows in mdc_map: 772
Sample DRG from mdc_map: '001'


In [456]:
# 1. Dictionary of the 'Missing 16' based on official CMS clinical groupings
manual_mdc_fix = {
    # Cardiology (MDC 05)
    '246': '05', '247': '05', '222': '05', '225': '05', '226': '05', '227': '05',
    # Orthopedics (MDC 08)
    '453': '08', '454': '08', '455': '08', '459': '08', '460': '08',
    # Neurology (MDC 01)
    '077': '01', '078': '01',
    # Digestive/Hepatobiliary (MDC 06/07)
    '339': '07', '342': '07', '343': '07'
}

# 2. Fill the NaNs using the map
df['MDC_Code'] = df['MDC_Code'].fillna(df['DRG_Code'].map(manual_mdc_fix))

print(f"Final Audit: {df['MDC_Code'].isna().sum()} rows remaining unmapped.")

Final Audit: 0 rows remaining unmapped.


In [457]:
df.head()

,Provider_ID,Hospital_Name,City,Address,State_Code,Zip,State,Rural_Urban_Code,Region_Type,DRG_Code,DRG_Description,Total_Discharges,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt,MDC_Code
0,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",003,"ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DIAGNOSIS EXCEPT FACE, MOUTH AND NEC",14,663764.35714,120219.928570,115544.142860,00
1,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE COMPLEX CNS PRINCIPAL DIAGNOSIS WITH MCC O,26,180980.88462,37321.038462,35261.807692,1
2,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE COMPLEX CNS PRINCIPAL DIAGNOSIS WITHOUT MC,12,105824.33333,26936.666667,25048.916667,1
3,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",025,CRANIOTOMY AND ENDOVASCULAR INTRACRANIAL PROCEDURES WITH MCC,16,242539.50000,34745.375000,32438.625000,1
4,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",038,EXTRACRANIAL PROCEDURES WITH CC,11,122741.18182,14999.818182,9579.363636,1


In [458]:
import pandas as pd
import duckdb

# 1. Apply your reliable mapping
full_25_mdc_mapping = {
    '00': 'High-Complexity/Transplant', '01': 'Neurology', '02': 'Ophthalmology',
    '03': 'ENT', '04': 'Pulmonology', '05': 'Cardiology',
    '06': 'Gastroenterology', '07': 'Hepatobiliary', '08': 'Orthopedics',
    '09': 'Dermatology', '10': 'Endocrinology', '11': 'Nephrology',
    '12': 'Urology', '13': 'Gynecology', '14': 'Obstetrics',
    '15': 'Neonatology', '16': 'Hematology', '17': 'Oncology',
    '18': 'Infectious Disease', '19': 'Psychiatry', '20': 'Substance Abuse',
    '21': 'Trauma/Toxicology', '22': 'Burn Center', 
    '23': 'Rehabilitation/Aftercare',  # ← Changed from 'General Medicine'
    '24': 'Polytrauma', '25': 'HIV/AIDS Care'
}

# 2. Robust Column Creation
# We convert to string and zfill(2) to ensure '5' becomes '05'
df['Service_Line'] = df['MDC_Code'].astype(str).str.strip().str.zfill(2).map(full_25_mdc_mapping)

# 3. Default for anything that didn't match (Safety Net)
df['Service_Line'] = df['Service_Line'].fillna('General Med/Surg')

print("Mapping applied. Strategy CSV generated for Power BI.")

Mapping applied. Strategy CSV generated for Power BI.


In [459]:
df.head(1)

,Provider_ID,Hospital_Name,City,Address,State_Code,Zip,State,Rural_Urban_Code,Region_Type,DRG_Code,DRG_Description,Total_Discharges,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt,MDC_Code,Service_Line
0,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",003,"ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DIAGNOSIS EXCEPT FACE, MOUTH AND NEC",14,663764.35714,120219.92857,115544.14286,00,High-Complexity/Transplant


In [461]:
df.head()

,Provider_ID,Hospital_Name,City,Address,State_Code,Zip,State,Rural_Urban_Code,Region_Type,DRG_Code,DRG_Description,Total_Discharges,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt,MDC_Code,Service_Line
0,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",003,"ECMO OR TRACHEOSTOMY WITH MV >96 HOURS OR PRINCIPAL DIAGNOSIS EXCEPT FACE, MOUTH AND NEC",14,663764.35714,120219.928570,115544.142860,00,High-Complexity/Transplant
1,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",023,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE COMPLEX CNS PRINCIPAL DIAGNOSIS WITH MCC O,26,180980.88462,37321.038462,35261.807692,1,Neurology
2,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",024,CRANIOTOMY WITH MAJOR DEVICE IMPLANT OR ACUTE COMPLEX CNS PRINCIPAL DIAGNOSIS WITHOUT MC,12,105824.33333,26936.666667,25048.916667,1,Neurology
3,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",025,CRANIOTOMY AND ENDOVASCULAR INTRACRANIAL PROCEDURES WITH MCC,16,242539.50000,34745.375000,32438.625000,1,Neurology
4,010001,Southeast Health Medical Center,Dothan,1108 Ross Clark Circle,01,36301,AL,2,"Metropolitan area high commuting: primary flow 30% or more to a urbanized area of 50,000 and greater",038,EXTRACRANIAL PROCEDURES WITH CC,11,122741.18182,14999.818182,9579.363636,1,Neurology


# Feature Adding

In [463]:
df['Avg_Patient_OOP'] = df['Avg_Tot_Pymt_Amt'] - df['Avg_Mdcr_Pymt_Amt']
df['Patient_Share_Pct'] = ((df['Avg_Patient_OOP'] / df['Avg_Tot_Pymt_Amt']) * 100)
df['Patient_Share_Pct'] = df['Patient_Share_Pct'].fillna(0)

print("New RCM Metrics Added:")
df[['Avg_Tot_Pymt_Amt', 'Avg_Mdcr_Pymt_Amt', 'Avg_Patient_OOP', 'Patient_Share_Pct']].head()

New RCM Metrics Added:


,Avg_Tot_Pymt_Amt,Avg_Mdcr_Pymt_Amt,Avg_Patient_OOP,Patient_Share_Pct
0,120219.928570,115544.142860,4675.785710,3.889360
1,37321.038462,35261.807692,2059.230770,5.517614
2,26936.666667,25048.916667,1887.750000,7.008105
3,34745.375000,32438.625000,2306.750000,6.639013
4,14999.818182,9579.363636,5420.454546,36.136802


In [582]:
df['Reimbursement_Rate_Pct'] = (df['Avg_Mdcr_Pymt_Amt'] / df['Avg_Submtd_Cvrd_Chrg']) * 100

display(df[['Hospital_Name', 'DRG_Code', 'Avg_Submtd_Cvrd_Chrg', 'Avg_Tot_Pymt_Amt', 'Reimbursement_Rate_Pct']].head())

,Hospital_Name,DRG_Code,Avg_Submtd_Cvrd_Chrg,Avg_Tot_Pymt_Amt,Reimbursement_Rate_Pct
0,Southeast Health Medical Center,003,663764.35714,120219.928570,17.407404
1,Southeast Health Medical Center,023,180980.88462,37321.038462,19.483719
2,Southeast Health Medical Center,024,105824.33333,26936.666667,23.670281
3,Southeast Health Medical Center,025,242539.50000,34745.375000,13.374574
4,Southeast Health Medical Center,038,122741.18182,14999.818182,7.804523


In [583]:
df.to_csv('Hospital_RCM_Cleaned_Data.csv', index=False)

# SQL Analysis 

In [470]:
pd.set_option('display.max_columns', None) 

pd.set_option('display.width', 1000)

pd.set_option('display.max_colwidth', None)

## Top Service Lines/Medical Dept. With The Most Revenue Share ##

In [491]:
top_5_global_share_query = """
SELECT 
    COALESCE(Service_Line, 'Unmapped/Other') AS "Service_Line",
    SUM(Total_Discharges) AS "Total_Discharges",
    ROUND(SUM(Total_Discharges * Avg_Tot_Pymt_Amt), 2) AS "Total_Revenue",
    -- This line calculates share against the WHOLE dataset, not just the Top 5
    ROUND(
        (SUM(Total_Discharges * Avg_Tot_Pymt_Amt) / (SELECT SUM(Total_Discharges * Avg_Tot_Pymt_Amt) FROM df)) * 100, 
        2
    ) AS "Revenue_Share_Percent"
FROM df
GROUP BY Service_Line
ORDER BY "Total_Revenue" DESC
LIMIT 5;
"""

top_5_global_df = duckdb.query(top_5_global_share_query).to_df()
print(top_5_global_df)

top_5_global_df.to_csv('Top_5_Service_Line_Revenue.csv', index=False)

Top 5 Service Lines and their Overall Hospital Revenue Share:
         Service_Line  Total_Discharges  Total_Revenue  Revenue_Share_Percent
0          Cardiology         1140658.0   2.183770e+10                  24.70
1  Infectious Disease          793556.0   1.636278e+10                  18.51
2         Orthopedics          454277.0   9.929639e+09                  11.23
3         Pulmonology          656325.0   9.222840e+09                  10.43
4           Neurology          389866.0   6.696127e+09                   7.57


## Top states by Revenue Share ##

In [493]:

state_revenue_share_query = """
WITH Total_National_Revenue AS (
    SELECT SUM(Total_Discharges * Avg_Tot_Pymt_Amt) as national_sum FROM df
)
SELECT 
    State,
    ROUND(SUM(Total_Discharges * Avg_Tot_Pymt_Amt) / 1000000000.0, 2) AS "Total_Revenue_Billions",
    ROUND((SUM(Total_Discharges * Avg_Tot_Pymt_Amt) / (SELECT national_sum FROM Total_National_Revenue)) * 100, 2) AS "Revenue_Share_Percent"
FROM df
GROUP BY State
ORDER BY "Total_Revenue_Billions" DESC
LIMIT 5;
"""

top_5_states_df = duckdb.query(state_revenue_share_query).to_df()

# Display results
print("Top 5 States by Revenue Share:")
print(top_5_states_df)

# Save for Power BI
top_5_states_df.to_csv('Top_5_States_Revenue_Share.csv', index=False)

Top 5 States by Revenue Share:
  State  Total_Revenue_Billions  Revenue_Share_Percent
0    CA                   10.31                  11.66
1    NY                    7.76                   8.78
2    FL                    6.35                   7.18
3    TX                    5.70                   6.45
4    PA                    3.92                   4.44


## Revenue Concentration Anslysis by DRG ##

In [556]:
sql_query = """
WITH DRG_Revenue AS (
    SELECT 
        DRG_Description, 
        SUM(Total_Discharges * Avg_Tot_Pymt_Amt) AS Procedure_Revenue
    FROM df
    WHERE DRG_Description IS NOT NULL
    GROUP BY DRG_Description
),
DRG_Ranking AS (
    SELECT 
        DRG_Description,
        Procedure_Revenue,
        NTILE(20) OVER (ORDER BY Procedure_Revenue DESC) AS Five_Percent_Bucket
    FROM DRG_Revenue
),
Bucket_Aggregation AS (
    -- Step 3: Summarize Revenue by Bucket
    SELECT 
        Five_Percent_Bucket,
        (Five_Percent_Bucket * 5) || '% Procedures' AS Portfolio_Segment,
        SUM(Procedure_Revenue) AS Bucket_Revenue
    FROM DRG_Ranking
    GROUP BY Five_Percent_Bucket
),
Global_Total AS (
    SELECT SUM(Bucket_Revenue) AS Grand_Total FROM Bucket_Aggregation
)
-- Step 4: Final Output with Cumulative Revenue for the X-Axis
SELECT 
    Portfolio_Segment,
    Five_Percent_Bucket,
    ROUND(Bucket_Revenue, 2) AS Revenue_In_Bucket,
    ROUND((Bucket_Revenue / (SELECT Grand_Total FROM Global_Total)) * 100, 2) AS Bucket_Contribution_Pct,
    ROUND(SUM(Bucket_Revenue) OVER (ORDER BY Five_Percent_Bucket ASC) / (SELECT Grand_Total FROM Global_Total) * 100, 2) AS Cumulative_Revenue_Pct
FROM Bucket_Aggregation
ORDER BY Five_Percent_Bucket ASC
Limit 5;
"""

# Execute and Preview
concentration_df = duckdb.query(sql_query).to_df()
display(concentration_df)
concentration_df.to_csv('Revenue_concentration_by_DRG.csv', index=False)

,Portfolio_Segment,Five_Percent_Bucket,Revenue_In_Bucket,Bucket_Contribution_Pct,Cumulative_Revenue_Pct
0,5% Procedures,1,4.593886e+10,51.96,51.96
1,10% Procedures,2,1.582456e+10,17.90,69.86
2,15% Procedures,3,8.479763e+09,9.59,79.45
3,20% Procedures,4,5.411963e+09,6.12,85.57
4,25% Procedures,5,3.652817e+09,4.13,89.70


## Medical Complexity(MCC,CC) Level Analysis ##

In [580]:
query = """
WITH drg_raw AS (
    SELECT 
        DRG_Code,
        DRG_Description,
        CASE 
            WHEN DRG_Description LIKE '%W MCC%' OR DRG_Description LIKE '%WITH MCC%' THEN 'MCC'
            WHEN DRG_Description LIKE '%W CC%' OR DRG_Description LIKE '%WITH CC%' THEN 'CC'
            WHEN DRG_Description LIKE '%W/O CC/MCC%' OR 
                 DRG_Description LIKE '%WITHOUT CC/MCC%' OR
                 DRG_Description LIKE '%W/O MCC%' THEN 'No CC/MCC'
            ELSE 'Other'
        END AS Complexity_Level,
        Total_Discharges,
        (Avg_Tot_Pymt_Amt * Total_Discharges) AS Line_Revenue,
        (Avg_Patient_OOP * Total_Discharges) AS Line_Patient_OOP
    FROM df
    WHERE DRG_Description IS NOT NULL
)
SELECT 
    Complexity_Level,
    COUNT(DISTINCT DRG_Code) AS Num_DRGs,
    SUM(Total_Discharges) AS Total_Cases,
    -- True Weighted Average Payment
    ROUND(SUM(Line_Revenue) / SUM(Total_Discharges), 2) AS Avg_Payment_Per_Case,
    -- True Weighted Average Patient Out-of-Pocket
    ROUND(SUM(Line_Patient_OOP) / SUM(Total_Discharges), 2) AS Avg_Patient_OOP_Per_Case,
    ROUND(SUM(Line_Revenue), 2) AS Total_Revenue,
    -- Revenue Share Percentage
    ROUND(SUM(Line_Revenue) * 100.0 / SUM(SUM(Line_Revenue)) OVER (), 2) AS Revenue_Share_Pct
FROM drg_raw
GROUP BY Complexity_Level
ORDER BY Avg_Payment_Per_Case DESC;
"""

mcc_cc_other_comp = duckdb.query(query).to_df()
display(mcc_cc_other_comp)
mcc_cc_other_comp.to_csv('Mcc_cc_other_comp.csv', index=False)

,Complexity_Level,Num_DRGs,Total_Cases,Avg_Payment_Per_Case,Avg_Patient_OOP_Per_Case,Total_Revenue,Revenue_Share_Pct
0,MCC,170,2503001.0,19213.11,2988.29,4.809044e+10,54.39
1,Other,133,1324944.0,18490.76,3221.42,2.449922e+10,27.71
2,No CC/MCC,83,167938.0,14754.14,3267.35,2.477782e+09,2.80
3,CC,148,964442.0,13835.63,2822.08,1.334366e+10,15.09


## Count of Underpaid Hospital of Different Payment Gap Thresholds From The State AVG ##

In [505]:
sensitivity_query = """
WITH State_DRG_Benchmarks AS (
    SELECT 
        State, 
        DRG_Code, 
        AVG(Avg_Tot_Pymt_Amt) AS State_Avg
    FROM df
    GROUP BY State, DRG_Code
),
Hospital_Variances AS (
    SELECT 
        df.Provider_ID,
        df.Total_Discharges,
        df.Avg_Tot_Pymt_Amt AS Hospital_Payment,
        s.State_Avg,
        -- Percentage Gap
        ((df.Avg_Tot_Pymt_Amt - s.State_Avg) / s.State_Avg) * 100 AS Variance_Pct,
        -- Dollar Loss: (State Average - What we got) * Number of Patients
        ABS(s.State_Avg - df.Avg_Tot_Pymt_Amt) * df.Total_Discharges AS Individual_Loss_Dollars
    FROM df
    JOIN State_DRG_Benchmarks s 
      ON df.State = s.State 
      AND df.DRG_Code = s.DRG_Code
)
-- We use a UNION to create the sensitivity table for Power BI
SELECT '30% Less' AS Threshold, 30 AS Sort_Order, COUNT(DISTINCT Provider_ID) AS Hospital_Count, ROUND(SUM(Individual_Loss_Dollars)/1000000, 2) AS Potential_Loss_Millions FROM Hospital_Variances WHERE Variance_Pct <= -30
UNION ALL
SELECT '40% Less' AS Threshold, 40 AS Sort_Order, COUNT(DISTINCT Provider_ID) AS Hospital_Count, ROUND(SUM(Individual_Loss_Dollars)/1000000, 2) AS Potential_Loss_Millions FROM Hospital_Variances WHERE Variance_Pct <= -40
UNION ALL
SELECT '50% Less' AS Threshold, 50 AS Sort_Order, COUNT(DISTINCT Provider_ID) AS Hospital_Count, ROUND(SUM(Individual_Loss_Dollars)/1000000, 2) AS Potential_Loss_Millions FROM Hospital_Variances WHERE Variance_Pct <= -50
UNION ALL
SELECT '60% Less' AS Threshold, 60 AS Sort_Order, COUNT(DISTINCT Provider_ID) AS Hospital_Count, ROUND(SUM(Individual_Loss_Dollars)/1000000, 2) AS Potential_Loss_Millions FROM Hospital_Variances WHERE Variance_Pct <= -60
UNION ALL
SELECT '70% Less' AS Threshold, 70 AS Sort_Order, COUNT(DISTINCT Provider_ID) AS Hospital_Count, ROUND(SUM(Individual_Loss_Dollars)/1000000, 2) AS Potential_Loss_Millions FROM Hospital_Variances WHERE Variance_Pct <= -70
ORDER BY Sort_Order ASC
"""

sensitivity_results = duckdb.query(sensitivity_query).to_df()
sensitivity_results.to_csv('Hospital_Loss_Sensitivity.csv', index=False)

display(sensitivity_results)

AUDIT SUMMARY: HOSPITAL COUNTS AND REVENUE LOSS


,Threshold,Sort_Order,Hospital_Count,Potential_Loss_Millions
0,30% Less,30,728,701.29
1,40% Less,40,242,164.45
2,50% Less,50,72,37.55
3,60% Less,60,20,8.54
4,70% Less,70,3,2.44


## Num of Hospitals and Potential Loss Getting 30% Less Payment than State Avg on Same DRG ##

In [565]:
threshold_percent = -30

state_underperformer_query = f"""
WITH State_DRG_Benchmarks AS (
    -- Step 1: Establish the 'Peer Average' for every DRG in every State
    SELECT 
        State, 
        DRG_Code, 
        AVG(Avg_Tot_Pymt_Amt) AS State_Avg
    FROM df
    GROUP BY State, DRG_Code
),
Hospital_Analysis AS (
    -- Step 2: Compare each hospital to its specific State/DRG peer group
    SELECT 
        df.Provider_ID,
        df.State,
        ((df.Avg_Tot_Pymt_Amt - s.State_Avg) / s.State_Avg) * 100 AS Variance_Pct,
        (s.State_Avg - df.Avg_Tot_Pymt_Amt) * df.Total_Discharges AS Lost_Revenue
    FROM df
    JOIN State_DRG_Benchmarks s 
      ON df.State = s.State 
      AND df.DRG_Code = s.DRG_Code
    WHERE ((df.Avg_Tot_Pymt_Amt - s.State_Avg) / s.State_Avg) * 100 <= {threshold_percent}
)
-- Step 3: Count unique hospitals and total lost revenue per State
SELECT 
    State,
    COUNT(DISTINCT Provider_ID) AS Underperformer_Hospital_Count,
    ROUND(SUM(Lost_Revenue) / 1000000, 2) AS Potential_Loss_Millions
FROM Hospital_Analysis
GROUP BY State
ORDER BY Underperformer_Hospital_Count DESC;
"""

state_summary_df = duckdb.query(state_underperformer_query).to_df()
print(f"Summary of Hospitals getting {abs(threshold_percent)}% LESS than State Peers:")
display(state_summary_df.head(10))

state_summary_df.to_csv('Underperformers_By_State.csv', index=False)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Summary of Hospitals getting 30% LESS than State Peers:


,State,Underperformer_Hospital_Count,Potential_Loss_Millions
0,CA,98,133.79
1,TX,84,48.03
2,NY,70,159.41
3,PA,50,23.12
4,IL,45,26.61
5,MD,36,136.79
6,FL,32,9.75
7,GA,26,24.74
8,NJ,25,10.66
9,OH,23,16.90


## State Wise DRG with the Most Payment Inconsistency ##

In [533]:
sql_query = """
SELECT 
    State, 
    DRG_Code,
    DRG_Description,
    COUNT(Provider_ID) as Hospital_Count,
    SUM(Total_Discharges) AS "Total Discharges" ,
    ROUND(AVG(Avg_Tot_Pymt_Amt), 2) as Avg_Payment,
    ROUND(STDDEV_SAMP(Avg_Tot_Pymt_Amt) / AVG(Avg_Tot_Pymt_Amt), 4) as CV_Score
FROM 
    df
GROUP BY 
     State, DRG_Code,DRG_Description
HAVING Hospital_Count>100
ORDER BY 
    CV_Score DESC;
"""

top_DRGs_with_payment_inconsistency = duckdb.query(sql_query).to_df()
display(top_DRGs_with_payment_inconsistency.head(20))

top_DRGs_with_payment_inconsistency.to_csv('Top_10_Inconsistent_DRGs.csv', index=False)


,State,DRG_Code,DRG_Description,Hospital_Count,Total Discharges,Avg_Payment,CV_Score
0,TX,641,"MISCELLANEOUS DISORDERS OF NUTRITION, METABOLISM, FLUIDS AND ELECTROLYTES WITHOUT MCC",123,3604.0,8632.95,1.2760
1,TX,690,KIDNEY AND URINARY TRACT INFECTIONS WITHOUT MCC,152,5429.0,8544.29,1.1684
2,TX,683,RENAL FAILURE WITH CC,139,5600.0,9416.13,1.1188
3,TX,065,INTRACRANIAL HEMORRHAGE OR CEREBRAL INFARCTION WITH CC OR TPA IN 24 HOURS,127,4844.0,10341.90,1.0006
4,TX,189,PULMONARY EDEMA AND RESPIRATORY FAILURE,144,6601.0,11790.62,0.8808
5,IL,291,HEART FAILURE AND SHOCK WITH MCC,108,16381.0,12232.13,0.8332
6,TX,640,"MISCELLANEOUS DISORDERS OF NUTRITION, METABOLISM, FLUIDS AND ELECTROLYTES WITH MCC",140,5831.0,12344.82,0.8315
7,TX,291,HEART FAILURE AND SHOCK WITH MCC,194,20482.0,11836.02,0.7594
8,TX,280,"ACUTE MYOCARDIAL INFARCTION, DISCHARGED ALIVE WITH MCC",151,6322.0,14406.81,0.7117
9,TX,481,HIP AND FEMUR PROCEDURES EXCEPT MAJOR JOINT WITH CC,122,4447.0,18274.45,0.6641


## Medical Departments With the Most Avg Patient Responsibility % in Reimbursement Compare to Overall Avg ##

In [526]:
sql_query = """
WITH National_Calculation AS (
    -- This creates the National Benchmark (The "Global" Average)
    SELECT 
        SUM(CAST(Total_Discharges AS DOUBLE) * Avg_Patient_OOP) AS Global_OOP_Total,
        SUM(CAST(Total_Discharges AS DOUBLE) * Avg_Tot_Pymt_Amt) AS Global_Reimbursement_Total
    FROM df
),
Benchmark_Rate AS (
    -- The National Responsibility % (The "Single Number" baseline)
    SELECT 
        ROUND((Global_OOP_Total / NULLIF(Global_Reimbursement_Total, 0)) * 100, 2) AS National_Patient_OOP_Rate
    FROM National_Calculation
)
SELECT 
    h.Service_Line,
    MAX(br.National_Patient_OOP_Rate) AS National_Patient_OOP_Rate,
    -- Service Line specific weighted percentage
    ROUND(
        (SUM(CAST(h.Total_Discharges AS DOUBLE) * h.Avg_Patient_OOP) / 
         NULLIF(SUM(CAST(h.Total_Discharges AS DOUBLE) * h.Avg_Tot_Pymt_Amt), 0)) * 100, 2
    ) AS Service_Line_Patient_OOP_Rate
FROM 
    df h, 
    Benchmark_Rate br
WHERE 
    h.Service_Line IS NOT NULL
GROUP BY 
    h.Service_Line
ORDER BY 
    Service_Line_Patient_OOP_Rate DESC;
"""

rcm_audit_results = duckdb.query(sql_query).to_df()

display(rcm_audit_results)
rcm_audit_results.to_csv('National_vs_ServiceLine_OOP_Rate.csv', index=False)

,Service_Line,National_Patient_OOP_Rate,Service_Line_Patient_OOP_Rate
0,Urology,16.99,29.58
1,Ophthalmology,16.99,27.90
2,ENT,16.99,26.92
3,Gynecology,16.99,23.31
4,Dermatology,16.99,23.04
5,Hepatobiliary,16.99,21.01
6,Oncology,16.99,20.75
7,Rehabilitation/Aftercare,16.99,20.48
8,Nephrology,16.99,20.26
9,Endocrinology,16.99,20.16
